In [ ]:
import numpy as np

In [ ]:
def pretty_print(ideal):
    for eq in ideal.gens():
        print(eq)

In [ ]:
class VectorOfMatrices:
    def __init__(self, matrices):
        self.matrices = matrices
        self.len = len(matrices)

    def get_len(self):
        return self.len
     
    def __mul__(self, other):
        # https://doc.sagemath.org/html/en/reference/matrices/sage/matrix/matrix_dense.html
        if not isinstance(other, sage.matrix.matrix_dense.Matrix_dense):
            raise TypeError("We only multiply by a matrix my dear")

        if other.ncols() != self.len:
            raise ValueError("Matrix dimensions do not match")

        # multiply them by hand
        result = [sum(other[i, j] * self.matrices[j] for j in range(self.len)) for i in range(other.nrows())]
        # gives back a list
        return result

    def __sub__(self, other):
        # https://doc.sagemath.org/html/en/reference/modules/sage/modules/vector_modn_dense.html#sage.modules.vector_modn_dense.Vector_modn_dense
        # if not isinstance(other, sage.modules.vector_modn_dense):
            # raise TypeError("We only subtract other VectorOfMatrices my dear")

        if other.len != self.len:
            raise ValueError("Vector dimensions do not match")

        result = [self.matrices[i] - other.matrices[i] for i in range(self.len)]
        return result
        
    def __repr__(self):
        return f"VectorOfMatrices(\n{self.matrices})"
    
    __rmul__ = __mul__
    __rsub__ = __sub__

In [201]:
class Instance:
    def __init__(self, n, m, p):
        self.n = n
        self.m = m
        self.size = p
        self.field = GF(p)

        self.R = PolynomialRing(self.field, n^2 + m^2, 'x')
        self.R.inject_variables(verbose=False)
        
        self.F = []
        self.S = np.zeros((self.n, self.n), dtype=int)
        self.T = np.zeros((self.m, self.m), dtype=int)
        
        self.P = []

        # print("--------------------------------------------------------------------------------------------------")
        # print(f"| An instance of inhQMLE over Finite Field with {self.size} elements, n = {self.n} variables and m = {self.m} equations. |")
        # print("--------------------------------------------------------------------------------------------------")

    def populate(self):
        self.generate_private()
        self.generate_public()
        
    def _random_symmetric_matrix(self):
        # get a random matrix
        A = random_matrix(self.field, self.n, self.n)
        # and make it symmetric
        return A + A.transpose()

    def _random_polynomial(self):
        return self._random_symmetric_matrix(), random_vector(self.field, self.n), self.field.random_element()

    def generate_private(self):
        self.T = matrix(GL(self.m, self.field).random_element())
        self.S = matrix(GL(self.n, self.field).random_element())
        if self.F == []:
            # print(f"Generating random polynomials over a finite field with {self.size} elements")
            self.F = [[],[],[]]
            for i in range(m):
                A, B, C = self._random_polynomial()
                self.F[0].append(A)
                self.F[1].append(B)
                self.F[2].append(C)
    
    def show_private(self):
        print(f"The input transformation S\n{self.S}")
        print(f"The output transformation T\n{self.T}")
        print("The central map F[i](x) = x^T A[i]x + b[i]x + c[i]")
        for i in range(m):
            print(f"F[{i}]:Matrix\n{self.F[0][i]}, Vector {self.F[1][i]}, Scalar {self.F[2][i]}\n---------------------------------------------")

    def generate_public(self):
        if self.P == []:
            self.P = [[],[],[]]
            v = VectorOfMatrices([self.S.transpose()*A*self.S for A in self.F[0]])
            self.P = [self.T*v, self.T*matrix([B*self.S for B in self.F[1]]), self.T*vector(self.F[2])]

    def show_public(self):
        print("The public set of polynomials P[i](x) = sum_{j=1}^m t_{i,j} f^{(j)}(Sx)")
        for i in range(m):
            print(f"P[{i}]: Matrix\n{self.P[0][i]}, Vector {self.P[1][i]}, Scalar {self.P[2][i]}\n---------------------------------------------")

    def model(self):
        gens = self.R.gens()

        # first n^2 elements of R2
        S_ = matrix(self.R, self.n, self.n, gens[:self.n^2])

        # we get the last m^2 variables of R2
        T_ = matrix(self.R, self.m, self.m, gens[-self.m^2:])

        auxiliary = [[],[],[]]
        
        auxiliary0 = T_*VectorOfMatrices(self.P[0])
        Q0 = VectorOfMatrices([S_.transpose()*A*S_ for A in self.F[0]])
        
        auxiliary1 = T_*self.P[1]
        Q1 = matrix([B*S_ for B in self.F[1]])
        
        auxiliary2 = T_*self.P[2]
        Q2 = vector(self.F[2])
        
        # add the field equations by hand
        system = [x^self.size - x for x in gens]
        M = VectorOfMatrices(auxiliary0) - Q0
        for i in range(m):
            system += M[i].list()
            system += (auxiliary1[i] - Q1[i]).list()
            system += [auxiliary2[i] - Q2[i]]
        
        # the list(set(.)) removes the duplicates
        return self.R.ideal(list(set(system)))

    # given a system of polynomials, check if it correctly corresponds
    # to the actual matrices it was created from
    #
    # ideal is an ideal of multivariate polynomial ring defined above
    def is_ideal_correct(self, ideal):

        # point = self.S.list() + matrix(GL(self.m, self.field).random_element()).list()
        point = self.S.list() + self.T.inverse().list()
        # point = self.S.list()+self.T.list()
    
        gens = self.R.gens()
        # make an appropriate dict
        substitution_dict = {gens[i]: point[i] for i in range(len(point))}
        
        return all(f.subs(substitution_dict) == 0 for f in ideal.gens())


    # i think this is wrong as the variety might have many points but 
    # we are only interested in the point corresponding to S and T
    def is_variety_correct(self, variety):
        point = self.S.list() + self.T.inverse().list()
        gens = self.R.gens()

        return all(variety[j][var] == point[i] for i, var in enumerate(gens) for j in range(len(variety)))
        # return all(f.subs(substitution_dict) == 0 for f in ideal.gens())

In [218]:
p = 101
n = 4
m = 3

inst = Instance(n, m, p)
inst.populate()
# inst.generate_private()
# inst.show_private()

# inst.generate_public()
# inst.show_public()

I = inst.model()
print(I)
print(inst.is_ideal_correct(I))
print(I.groebner_basis())
# %timeit I.groebner_basis()
# %timeit I.variety()
# for i in range(len(V)):
    # print(V[i])

# pt = inst.S.list() + inst.T.inverse().list()
# print(len(pt))
# for i, var in enumerate(inst.R.gens()):
#     if V[0][var] != pt[i]:
#         print(False)
    
# print(inst.is_variety_correct(V))

Ideal (-45*x19 + 9*x20 + 42*x21 + 50, 30*x0*x3 - 8*x3*x4 - 8*x0*x7 + 40*x4*x7 - 28*x3*x8 - 45*x7*x8 - 28*x0*x11 - 45*x4*x11 - 4*x8*x11 - 22*x3*x12 + 31*x7*x12 - 27*x11*x12 - 22*x0*x15 + 31*x4*x15 - 27*x8*x15 - x12*x15 - 46*x19 - 46*x20 + 50*x21, 30*x2*x3 - 8*x3*x6 - 8*x2*x7 + 40*x6*x7 - 28*x3*x10 - 45*x7*x10 - 28*x2*x11 - 45*x6*x11 - 4*x10*x11 - 22*x3*x14 + 31*x7*x14 - 27*x11*x14 - 22*x2*x15 + 31*x6*x15 - 27*x10*x15 - x14*x15 + 41*x19 + 44*x20 + 14*x21, -10*x1*x3 + 3*x3*x5 + 3*x1*x7 - 8*x5*x7 + 10*x3*x9 + 34*x7*x9 + 10*x1*x11 + 34*x5*x11 + 33*x9*x11 + 23*x3*x13 + 25*x7*x13 - 42*x11*x13 + 23*x1*x15 + 25*x5*x15 - 42*x9*x15 - 2*x13*x15 + 22*x16 - 2*x17 + 12*x18, x8^101 - x8, x11^101 - x11, x18^101 - x18, x22^101 - x22, -10*x3^2 + 6*x3*x7 - 8*x7^2 + 20*x3*x11 - 33*x7*x11 + 33*x11^2 + 46*x3*x15 + 50*x7*x15 + 17*x11*x15 - 2*x15^2 + 34*x16 + 19*x17 + 12*x18, x1^101 - x1, x9^101 - x9, x10^101 - x10, x20^101 - x20, -16*x2*x3 - 20*x3*x6 - 20*x2*x7 + 24*x6*x7 + 30*x3*x10 + 27*x7*x10 + 30*x2*x11 +

KeyboardInterrupt: 

In [212]:
primes = Primes()
p = primes.next(2)

i = 0
n = 5
m = 4

while (i < 5):
    i+=1
    inst = Instance(n, m, p)
    inst.populate()
    I = inst.model()
    if not inst.is_ideal_correct(I):
        print("oh nooooooo")

    print(timeit.eval("I.groebner_basis()"))
    inst = Instance(n, m, p^2)
    inst.populate()
    I = inst.model()
    if not inst.is_ideal_correct(I):
        print("oh nooooooo")

    print(timeit.eval("I.groebner_basis()"))
    
    inst = Instance(n, m, p^3)
    inst.populate()
    I = inst.model()
    if not inst.is_ideal_correct(I):
        print("oh nooooooo")

    print(timeit.eval("I.groebner_basis()"))

625 loops, best of 3: 293 ns per loop


KeyboardInterrupt: 